# Logs
Look at logs

In [ ]:
from __future__ import print_function
import glob
import os
import pandas as pd
from fahlogs import *

## Get Log files

In [ ]:
_pa = os.path.expanduser("~/server2/data/SVR2873114978/")
os.chdir(_pa)

In [ ]:
success_logs = glob.glob("PROJ9704/RUN*/CLONE*/results-???/log.txt")
fail_logs = glob.glob("PROJ9704/RUN*/CLONE*/0x*/results-???/log.txt")

## Make them into a dataframe

In [ ]:
df = pd.concat([
    pd.DataFrame([FAHLog(sl, success=True).to_dict() for sl in success_logs]),
    pd.DataFrame([FAHLog(fl, success=False).to_dict() for fl in fail_logs])
])
df['time'] = pd.to_datetime(df['time'], coerce=True)
df['finish_time'] = pd.to_datetime(df['finish_time'], coerce=True)
df['duration'] = df['finish_time'] - df['time']
df = df.sort('time')
df.tail()

In [ ]:
df[df['success']].tail()

## Plot WU duration

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
duration_hours = df['duration'].dropna() / np.timedelta64(1, 'h')
_ = plt.hist(duration_hours, bins=10)

## Do nifty aggregation on the dataframe

In [ ]:
sumdf = df.groupby('name').agg(['sum', 'count'])[['success']]
sumdf['percent'] = sumdf['success', 'sum'] / sumdf['success', 'count']
sumdf.sort('percent')

## Operating Systems

In [ ]:
df.os.value_counts()

In [ ]:
df.os.unique()

In [ ]:
for os in df.os.unique():
    print(os)
    print("------------")
    print("%r" % df[df['os'] == os].os2.value_counts())
    print()

### Windows-specific analysis

In [ ]:
windf = df[df['os'] == 'Windows']
sumdf = windf.groupby('name').agg(['sum', 'count'])[['success']]
sumdf['percent'] = sumdf['success', 'sum'] / sumdf['success', 'count']
sumdf.sort('percent')